# Polygenic Risk Scores Combinations
The Polygenic Risk Score Combinations Project Version 1.0

Follow this Jupyter Notebook step-by-step to replicate our results.

### 1. Setup config.py File
1. Set parent directories
2. Name file names of your input files
    - Input files should be PRSKB output, adnimerge and dxsum
3. Define columns of interest for your DataFrames
    - All TSV columns listed are mandatory
    - ADNI columns 'RID' and 'PTID' are mandatory
    - All DX columns are mandatory

### 2. Import Packages

In [ ]:
from config import tsv_path, tsv_cols, adni_path, adni_cols, dx_path, dx_cols
import pandas as pd
import numpy as np
from scipy.stats import gmean, hmean, ranksums, chisquare
import matplotlib.pyplot as plt

### 3. Initialize DataFrames and Global Variables

In [ ]:
tsv_df = pd.read_csv(tsv_path, header=0, usecols=tsv_cols,sep='\t')
adni_df = pd.read_csv(adni_path, header=0,usecols=adni_cols)
dx_df = pd.read_csv(dx_path,header=0,usecols=dx_cols)
demographics = ['PTID','AGE','PTGENDER','PTRACCAT','APOE4','DIAGNOSIS','EXAMDATE']

### 4. Create Filtered Dataframe for Reported Trait
- tsv_df will be the input dataframe
- traits arg example: 'Alzheimer|Dementia'
- Separate multiple traits by a pipe operator
- Example usage filtered_df = filter_df(tsv_df,'Liver|Hepatic|Blood')

In [ ]:
def filter_df(df,traits):
    df = df[df['Reported Trait'].str.contains(f'{traits}', case=False)].reset_index(drop=True)
    return df
filtered_df = filter_df(tsv_df,'Alzheimer|Dementia')
display(filtered_df.head(20))

### 5. Find the Earliest or Latest Diagnosis for a Patient in ADNI
- df is dx_df
- Options are 'Earliest' or 'Latest' EXAMDATE. For example, 'Earliest' would be 02/25/2006 andd 'Latest' would be 06/17/2011

In [ ]:
def find_diagnosis(df,options):
    # Define options
    if options == 'Earliest':
        keep = 'first'
        fill = 'bfill'
    elif options == 'Latest':
        keep = 'last'
        fill = 'ffill'
    else: 
        raise ValueError("Enter 'Earliest' or 'Latest'")

    # Fill empty values depending on option selected
    df['EXAMDATE'] = df['EXAMDATE'].fillna(method=fill)
    df['DIAGNOSIS'] = df['DIAGNOSIS'].fillna(method=fill)

    # Keep only the earliest or latest diagnosis depending on the option selected
    df = df.drop_duplicates(subset='RID', keep=keep).reset_index(drop=True)

    return df

diagnoses = find_diagnosis(dx_df,'Latest') # Example usage
display(diagnoses.head(20))

### 6. Merge Three DataFrames to Clean the Data
- demographics must be a list datatype.
- df1 = adni_df
- df2 = diagnoses. This must be generated from find_diagnosis function
- df3 = tsv_df
- You can refine the indices using the demographics argument. This selects demographics from ADNI, but you can only select those loaded in the original config.py file.

In [ ]:
def create_merged_df(df1,df2,df3,demographics):
    # demographics must be a list variable of demographics from adni_df
    # df1 = adni_df
    # df2 = diagnoses. This must be generated from find_diagnosis function
    # df3 = filtered_df
    # Drops duplicates in adni_df so that we only have a person's demographics once in the merged dataframe
    df1 = df1.drop_duplicates(subset='RID', keep='first').reset_index(drop=True)
    # Create Merged DataFrame on RID
    adni_diagnoses = pd.merge(df1,df2, left_on='RID',right_on='RID',how='inner').reset_index(drop=True)
    # Make sure the case is the same for each DataFrame, otherwise you will delete some of the samples
    adni_diagnoses['PTID'] = adni_diagnoses['PTID'].str.upper()
    df3['Sample'] = df3['Sample'].str.upper()
    # Create Merged DataFrame on Sample
    diagnoses_tsv = pd.merge(adni_diagnoses,df3, left_on='PTID', right_on='Sample', how='inner').reset_index(drop=True)
    #Create Pivot Table to Map Sample to their Percentile Scores
    merged_df = diagnoses_tsv.pivot_table(index=demographics, columns='Study ID', values='Percentile',aggfunc='first')
    
    return merged_df

merged = create_merged_df(adni_df,diagnoses,filtered_df,demographics=demographics)
display(merged.head(20))

### 7. Convert Range PRS to Mean of Lower and Upper Bounds
- Takes merged as df argument
- Converts UK Biobank Percentile Ranges, like 50-72 to numeric: 61.0
- This is a key step for future analysis steps, such as the means or Chi-squared tests

In [ ]:
def range_to_numeric(df):
    # Converts UK Biobank Percentile Ranges, like 50-72 to numeric: 61.0 
    # Input df should be the merged df, but could also be tsv_df
    def convert_value(x):
        if pd.isna(x):
            return None
        elif '-' in x:
            lower, upper = map(float, x.split('-'))
            return (lower + upper) / 2
        else:
            return float(x)

    # Apply the conversion to all columns in the DataFrame
    return df.applymap(convert_value)

numeric_prs = range_to_numeric(merged)
display(numeric_prs.head(20))

### 8. Drop Genome-Wide Association Studies
- Drop Genome-Wide Association Studies (GWAS) according to your preferred method
- In our study, we set unique to 50, meaning we got rid of GWAS that computed 50 or less unique PRS across the samples
- This resulted in 19 GWAS for analysis

In [ ]:
def drop_gwas(df,methods,threshold=0,gwas=None,unique=0):
    if gwas == None:
        gwas = [] # Initialize gwas as an empty list

    # Drop GWAS with preferred method
    if methods == 'dropna':
        # drop GWAS with NaN PRS values for any individuals
        df = df.dropna(axis='columns',how='any')
    elif methods == 'dropnaX':
        # drop GWAS that have X number of NaN PRS values
        df = df.dropna(axis='columns',thresh=threshold)
    elif methods == 'repeated':
        # Calculate the proportion of unique values in each column
        unique_proportion = df.apply(lambda col: col.nunique() / col.count(), axis='rows')
        # Filter columns where the proportion of unique values in a column is less than or equal to 
        # We would expect unique to equal 100 since PRS are percentiles. This means 100/808 is the expected unique proportion.
        # Set the appropriate cutoff for your needs. We used unique = 50
        del_cols = unique_proportion[unique_proportion <= (unique/len(df))].index
        # Drop columns with poor uniqueness
        df = df.drop(del_cols, axis='columns')
    elif methods == 'select':
        # removes custom GWAS that you select
        # gwas must be a list of GWAIDs. Ex: ['GCST000001','GCST000006']
        df = df.drop(gwas,axis='columns')
    else:
        raise ValueError("Enter specified method")

    return df
cleaned_df = drop_gwas(numeric_prs,'dropna')
unique_gwas = drop_gwas(cleaned_df,'repeated',unique=50)
display(unique_gwas.head(20))

### 9. Calculate Means 
- Calculate means of UK Biobank Percentile Polygenic Risk Scores (PRS) for each individual.
- df must be numeric_prs. The mean functions only take numeric datatypes.
- method can be mean for arithmetic mean, gmean for geometric mean, or hmean for harmonic mean. See numpy and scipy stats docs for more details.

In [ ]:
def means_calculations(df):
    # Remove all columns with NaN valuea and replace all UK Biobank 0th percentile scores with 1 so that .gmean() and .hmean() functions work
    df.dropna(axis='columns',how='any',inplace=True)
    df.replace(0.0, 1, inplace=True)
    # # Calculate Arithmetic, Geometric, and Harmonic Means Using .apply function. Add new columns with results
    df['Arithmetic Mean'] = df.apply(np.mean, axis=1)
    df['Geometric Mean'] = df.apply(gmean, axis=1)
    df['Harmonic Mean'] = df.apply(hmean, axis=1)

    return df

means_df = means_calculations(unique_gwas)
display(means_df.head(20))

### 10. Simple Diagnosis for Cases and Controls
- Merges definitions for cases into one name.
- In our study a case is cognitive impairment, "CI" and a control is cognitive normal, "CN"

In [ ]:
def simpledx(df):
    # Turning all indices into columns
    df.reset_index(inplace=True)
    # Changes DIAGNOSIS of Dementia or MCI to CI, which means Cognitive Impairment
    df['DIAGNOSIS'] = df['DIAGNOSIS'].replace(["Dementia", "MCI"], "CI")
    df = df.reset_index(drop=True)
    return df

final_df = simpledx(means_df)
display(final_df.head(20))

### 11. Mann-Whitney U Test
- Non-parametric T test used to determine if polygenic risk scores can discern between cases and controls.
- Arguments: simple_diagnosis DataFrame and GWAS to run the test on. Default is all GWAS and means.
- Output: mannwhitney DataFrame

In [ ]:
def mannwhitneyu(df):
  ## Performs Mann-Whitney U test on each column between 'CN' and 'CI' diagnosis groups
  # Args: The pandas dataframe simple diagnosis
  # Returns: A DataFrame with columns 'Columns','Statistic',and 'P-value' for each test
  results_df = pd.DataFrame(columns=['Columns', 'Statistic', 'P-value'])

  default = [col for col in df.columns if col not in demographics]
  for column in default:
    try:
      group1 = df[df['DIAGNOSIS'] == 'CN'][column].dropna()
      group2 = df[(df['DIAGNOSIS'] == 'CI')][column].dropna()
      statistic, p_value = ranksums(group1, group2)
    except ValueError:
      statistic, p_value = np.nan, np.nan  # Use np.nan for missing values

    results_df = pd.concat([results_df,pd.DataFrame({'Columns': [column], 'Statistic': [statistic], 'P-value': [p_value]})], ignore_index=True)

  return results_df

mannwhitney = mannwhitneyu(final_df)
display(mannwhitney)

### 12. Chi-Squared Test
- Test of independence used to determine if polygenic risk scores can discern between cases and controls in the top quintile of risk.
- Arguments: simple_diagnosis DataFrame, quantile cutoff, and GWAS to run the test on. Default is all GWAS and means. Quantile in our case was 0.8, meaning we looked at the top 0.2 PRS.
- Output: chisq DataFrame

In [ ]:
def chi_squared(df,quantile):
    #quantile is the lower cutoff for risk. If you pick 0.8, you will calculate Chi-squared for the top 20% of PRS
    results_df = pd.DataFrame(columns=['Columns','Statistic','P-value'])

    default = [col for col in df.columns if col not in demographics]
    for column in default:
        df_sorted = df.sort_values(by=column,ascending = False)
    
        cn_count = sum(df["DIAGNOSIS"] == 'CN')
        ci_count = sum(df["DIAGNOSIS"] == 'CI')

        threshold = int((1-quantile) * len(df_sorted))
        observed_cn = sum(df_sorted["DIAGNOSIS"].iloc[:threshold] == 'CN')
        observed_ci = sum(df_sorted["DIAGNOSIS"].iloc[:threshold] == 'CI')
        observed = np.array([observed_cn, observed_ci])

        expected = np.array([(cn_count/len(df_sorted)),(ci_count/len(df_sorted))]) * np.sum(observed)
        chisq_result, p_value = chisquare(observed, expected)
        
        temp_df = pd.DataFrame({'Columns': [column], 'Statistic': [chisq_result], 'P-value': [p_value]})

        # Concatenate results_df with temp_df
        results_df = pd.concat([results_df, temp_df], ignore_index=True)

    return results_df

chisq = chi_squared(final_df,0.8)
display(chisq)

### 13. Make Plots
- Make a Mann Whitney U Test Plot or a Chi-Squared test of independence plot
- Arguments: mannwhitney DataFrame for Mann-Whitney U Test plot, or chisq DataFrame for Chi-Squared plot. Second argument, options is the name of the plot.
- Name of the plot is either "Chi-Squared Test" or "Mann-Whitney U Test"


In [ ]:
def make_plots(df,options):
    #Sort P-Values so that bars increase in height with significance
    df_sorted = df.sort_values('P-value', ascending=False)

    # Extract sorted lists for plotting
    gwas_studies = df_sorted['Columns'].tolist()
    p_values = df_sorted['P-value'].tolist()

    # Calculate the negative log of the p-values
    neg_log_p_values = [-np.log10(p) if p > 0 else np.nan for p in p_values]  # Using nan for p-value of 0 (log(0) is undefined)

    # Create the bar graph
    plt.figure(figsize=(10, 5))
    colors = ['skyblue' for study in gwas_studies]
    bars = plt.bar(gwas_studies, neg_log_p_values, color=colors)

    # Add a line for the typical significance threshold (e.g., p < 0.05)
    significance_threshold = -np.log10(0.05)
    plt.axhline(y=significance_threshold, color='red', linestyle='--', label='Significance threshold (p=0.05)')

    # Annotate the bars with the actual p-values
    for bar, p_value in zip(bars, p_values):
        plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), f'p={p_value:.3g}', 
                ha='center', va='bottom', fontsize=6,rotation='horizontal')

    # Setting the labels and title
    plt.xlabel('GWAS Study', fontsize=12, fontweight='bold')
    plt.ylabel('-log10(p-value)', fontsize=12, fontweight='bold')
    plt.title(f'{options} Results', fontsize=14, fontweight='bold')
    plt.legend()

    # Rotate x-axis labels for better readability
    plt.xticks(rotation=90, fontsize=10, fontweight='bold')

    # Show the plot
    plt.tight_layout()  # Adjust layout to prevent overlap
    plt.show()

# Example Usage
chisq_plot = make_plots(chisq,"Chi-Squared Test")
mannwhitney_plot = make_plots(mannwhitney, "Mann-Whitney U Test")

### 14. Save Output
- Save final DataFrame 'final_df' as a CSV, TSV, or Excel (XLSX) file for viewing or further analysis
- Arguments: DataFrame, Filetype, and Path on your machine

In [ ]:
def save_output(df, option, path):
    # Options are 'CSV','TSV', or 'XLSX'
    if option == 'CSV':
        df.to_csv(path,index=True)
        print("CSV File Saved")
    elif option == 'TSV':
        df.to_csv(path,index=True,sep='\t')
        print("TSV File Saved")
    elif option == 'XLSX':
        df.to_excel(path,index=True)
        print("XLSX (Excel) File Saved")
    else: 
        print("Pick filetype option: 'CSV','TSV', or 'XLSX' ")

# final_csv = save_output(final_df,'CSV',path) # Example usage